In [1]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
import re
import time
from tqdm import tqdm

In [2]:
data = pd.DataFrame(columns=['title','content','link'])
data

,title,content,link


In [3]:
for page_num in tqdm(range(1,181)):   # 시사뉴스(111) : 1 ~ 180 page / 경제(115) : 1 ~ 40 page
    
    # 페이지 탐색
    page_url = f"https://kid.chosun.com/list_kj.html?catid=111&pn={page_num}"
    page_response = requests.get(page_url)
    page_html = page_response.content

    # BeautifulSoup을 사용하여 HTML 파싱
    page_soup = BeautifulSoup(page_html, 'html.parser')
    links = [link['href'] for link in page_soup.find_all('a')]
    links = Counter(links)
    links = [link for link in links if '/site/data/html_dir' in link]
    
    for news_link in links:
        time.sleep(0.08)
        # 웹 페이지의 HTML을 가져오기
        news_link = "https://kid.chosun.com"+news_link
        news_response = requests.get(news_link)
        news_content = news_response.content
        news_soup = BeautifulSoup(news_content, 'html.parser')

        # 기사 제목 추출
        title = news_soup.find('title').text.strip()
        title = re.sub(r'\[.*?\]','',title)
        # 기사 본문 추출
        news_content = []
        content = news_soup.find_all('div','Paragraph')
        for cont in content:
            c = cont.text
            except_cf = [dt.text for dt in cont.find_all('dt')] + [font.text for font in cont.find_all('font')] #+ [divv.text for divv in cont.find_all('div')] 
            if except_cf:
                for except_cfc in except_cf:
                    c = re.sub(re.escape(except_cfc), ' ', c)
            news_content.append(c.strip())

        #news_content = " ".join(news_content)
        if title and news_content:
            data = pd.concat([data,pd.DataFrame({'title': [title] , 'content' : [news_content] , 'link' : [news_link]})],ignore_index=True)
        


  0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [15:45<00:00,  5.25s/it]


In [4]:
data

,title,content,link
0,‘제주의 허파’ 선흘곶자왈은 멸종 위기·희귀 곤충들의 천국,"[바람과 물이 풍부하고, 화산 활동으로 돌이 다양한 제주. 특히 제주 선흘곶자왈은 ...",https://kid.chosun.com/site/data/html_dir/2023...
1,공적인 일에 종사해야 ‘公人’… 연예인에게도 ‘공인의 책임’ 있을까?,[최근 연예계는 사건·사고가 끊이지 않습니다. 지난달 23일 배우 이선균(48)이 ...,https://kid.chosun.com/site/data/html_dir/2023...
2,스트레스 받거나 우울·지루하면 ‘과식’ 성인뿐 아니라 4세 유아에게도 나타나,"[감정적 섭식, 음식을 통해 기분 전환하려는 행동을 '감정적 섭식(Emotional...",https://kid.chosun.com/site/data/html_dir/2023...
3,“돌고래에게 비좁은 수족관은 감옥” 벨루가는 바다로 돌아갈 수 있을까,"[생크추어리: 해양 동물을 보호하는 바다 쉼터, ""시급히 자연으로 돌려보냅시다!""지...",https://kid.chosun.com/site/data/html_dir/2023...
4,빌린 돈 미리 갚는데 수수료 내야 한다고?,[은행에서 빌린 돈을 원래 약속한 기간보다 빨리 갚으려고 할 때 돈을 더 내야 할 ...,https://kid.chosun.com/site/data/html_dir/2023...
...,...,...,...
1789,헬시 플레저,"['건강(Health)'과 '즐겁다(Pleasure)'가 합쳐진 단어로, 건강을 즐...",https://kid.chosun.com/site/data/html_dir/2022...
1790,"장애 가진 아기 기린, 보조 장치 차고 ‘폴짝’","[""폴짝폴짝, 이제 맘껏 뛰어놀래!""선천적으로 무릎 관절에 장애를 갖고 태어나 제대...",https://kid.chosun.com/site/data/html_dir/2022...
1791,"과일·채소 섭취, 어린이 ADHD 개선 효과",[과일과 채소를 많이 섭취하는 것이 어린이의 ADHD(주의력 결핍 과다 행동 장애)...,https://kid.chosun.com/site/data/html_dir/2022...
1792,가족愛 돈독할수록 자녀 잘 자란다,[부모·자녀 간 돈독한 유대 관계가 아이의 인생을 좌우할 수 있다는 연구 결과가 나...,https://kid.chosun.com/site/data/html_dir/2022...


In [5]:
data.to_csv("조선일보_시사.csv",index=False)